In [1]:
%load_ext lab_black
import subprocess, os
import pandas as pd
import glob, random

In [2]:
all_bam = {
    bam_path.split("/")[1].split(".trim")[0]: bam_path
    for bam_path in glob.glob("bam/*bam")
}
list(all_bam.items())[:3]

[('ATAC188_CB_FR062.1_ND_sB',
  'bam/ATAC188_CB_FR062.1_ND_sB.trimmed.bowtie2.filtered.bam'),
 ('ATAC088_CB_FR028.1_CVID_sB',
  'bam/ATAC088_CB_FR028.1_CVID_sB.trimmed.bowtie2.filtered.bam'),
 ('ATAC111_CB_FR036.1_NFKB1_sB',
  'bam/ATAC111_CB_FR036.1_NFKB1_sB.trimmed.bowtie2.filtered.bam')]

In [5]:
macs2_out = {}
for sample_label, bam in all_bam.items():
    subprocess.run(["mkdir", f"macs2_results/{sample_label}/"])
    macs2_out_prefix = f"macs2_results/{sample_label}/{sample_label}"
    macs2_out[sample_label] = macs2_out_prefix

    with open(f"macs2_call.log", "w") as log:
        macs2_cmd = [
            f"/home/murrelab/anaconda3/envs/hobj/bin/macs2",
            "callpeak",
            "-t",
            bam,
            "-g",
            "hs",
            "-f",
            "BAM",
            "-n",
            macs2_out_prefix,
            "-q",
            str(0.01),
            "-B",
            "--SPMR",
            "--keep-dup",
            "auto",
        ]

        macs2_callpeaks = subprocess.Popen(
            macs2_cmd,
            stderr=log,
        )
        macs2_callpeaks.wait()

    # peak number
    # subprocess.run(["wc", "-l", f"{macs2_out_prefix}_peaks.narrowPeak"])

In [16]:
summary = {
    "sample": [],
    "total tags": [],
    "dedup tags": [],
    "redundant rate": [],
    "num peaks": [],
}
for sample_label, _prefix in macs2_out.items():
    summary["sample"].append(sample_label)
    prefix = f"{_prefix}_peaks"
    count = sum(1 for line in open(f"{prefix}.narrowPeak"))
    summary["num peaks"].append(count)
    with open(f"{prefix}.xls") as stats:
        [stats.readline() for i in range(20)]
        summary["total tags"].append(int(stats.readline().split(": ")[1]))
        summary["dedup tags"].append(int(stats.readline().split(": ")[1]))
        stats.readline()
        summary["redundant rate"].append(float(stats.readline().split(": ")[1]))

pd.DataFrame(summary).to_csv("Summary_of_MACS2_Peak_Numbers.txt", sep="\t", index=None)

# Add ATAC292_BG_FR099.1_NFKB1_sB

In [2]:
sample_label = "ATAC292_BG_FR099.1_NFKB1_sB"
bam = "bam/" + sample_label + ".trimmed.bowtie2.filtered.bam"

In [3]:
!samtools index $bam

In [3]:
macs2_out = {}
subprocess.run(["mkdir", f"macs2_results/{sample_label}/"])
macs2_out_prefix = f"macs2_results/{sample_label}/{sample_label}"
macs2_out[sample_label] = macs2_out_prefix
with open(f"macs2_call.log", "a") as log:
    macs2_cmd = [
        f"/home/murrelab/anaconda3/envs/hobj/bin/macs2",
        "callpeak",
        "-t",
        bam,
        "-g",
        "hs",
        "-f",
        "BAM",
        "-n",
        macs2_out_prefix,
        "-q",
        str(0.01),
        "-B",
        "--SPMR",
        "--keep-dup",
        "auto",
    ]
    macs2_callpeaks = subprocess.Popen(
        macs2_cmd,
        stderr=log,
    )
    macs2_callpeaks.wait()
# peak number
# subprocess.run(["wc", "-l", f"{macs2_out_prefix}_peaks.narrowPeak"])

In [4]:
summary = {
    "sample": [],
    "total tags": [],
    "dedup tags": [],
    "redundant rate": [],
    "num peaks": [],
}
for sample_label, _prefix in macs2_out.items():
    summary["sample"].append(sample_label)
    prefix = f"{_prefix}_peaks"
    count = sum(1 for line in open(f"{prefix}.narrowPeak"))
    summary["num peaks"].append(count)
    with open(f"{prefix}.xls") as stats:
        [stats.readline() for i in range(20)]
        summary["total tags"].append(int(stats.readline().split(": ")[1]))
        summary["dedup tags"].append(int(stats.readline().split(": ")[1]))
        stats.readline()
        summary["redundant rate"].append(float(stats.readline().split(": ")[1]))

pd.DataFrame(summary).to_csv(
    "Summary_of_MACS2_Peak_Numbers.txt", sep="\t", index=None, mode="a"
)